In [ ]:
import itertools
from math import *

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from numpy.linalg import inv

from utils.colormap import truncate_colormap
import utils.itertools_recipes

In [ ]:
reds = np.random.multivariate_normal([5, 3], [[2, -1], [-1, 1]], 80)
blues = np.random.multivariate_normal([1, 0.5], [[5, 2], [2, 2]], 50)

In [ ]:
plt.scatter(*reds.T, c='r')
plt.scatter(*blues.T, c='b')